In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import re


## some config values 
embed_size = 300 # how big is each word vector
max_features = 120000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 72 # max number of words in a question to use


train = pd.read_csv("../input/train.csv")
# TO REMOVE
#train = train.head(10)

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }

def clean_contractions(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    return text


In [ ]:
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

def clean_special_chars(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text


In [ ]:
def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [ ]:
train["question_text"] = train["question_text"].str.lower()
train['question_text'] = train['question_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
train['question_text'] = train['question_text'].apply(lambda x: clean_numbers(x))
train['question_text'] = train['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
texts = train["question_text"].fillna("_##_").values
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=maxlen)

labels = train['target'].values

#Shuffling the data
np.random.seed(438)
permut = np.random.permutation(len(data))
data = data[permut]
labels = labels[permut]
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.3, random_state=42)

In [ ]:
def glove_matrix(word_index):
    embeddings_index = {}
    f = open('../input/embeddings/glove.840B.300d/glove.840B.300d.txt')
    for line in f:
        values = line.split(" ")
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    print('Found %s word vectors.' % len(embeddings_index))
    
    nb_words = min(max_features, len(word_index)+1)
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            if i >= max_features: continue
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

matrix_glove = glove_matrix(word_index)

In [ ]:
np.shape(matrix_glove)
#np.shape(matrix_para)

In [ ]:
from tqdm import tqdm

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = -0.0053247833,0.49346462
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index)+1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in tqdm(word_index.items()):
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix
matrix_para = load_para(word_index)

# Averaging the two embedding matrix
embedding_matrix = np.mean([matrix_glove, matrix_para], axis = 0)
np.shape(embedding_matrix)

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
# https://www.kaggle.com/suicaokhoailang/lstm-attention-baseline-0-652-lb
from keras.layers import Layer
from keras import initializers, regularizers, constraints

class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim


In [ ]:
from keras.layers import Embedding,Input,SpatialDropout1D,Bidirectional,CuDNNLSTM,CuDNNGRU,Dense,Flatten,Dropout,BatchNormalization
from keras.initializers import glorot_normal
from keras.models import Model

embedding_layer = Embedding(max_features,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)
sequence_input = Input(shape=(maxlen,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = SpatialDropout1D(0.1)(embedded_sequences)
x = Bidirectional(CuDNNLSTM(100, return_sequences=True, 
                                kernel_initializer=glorot_normal(seed=12300), recurrent_initializer='orthogonal'))(x)
x = Flatten()(x)
x = Dense(100, activation="relu", kernel_initializer=glorot_normal(seed=12300))(x)
x = Dropout(0.1)(x)
x = BatchNormalization()(x)
outp = Dense(1, activation="sigmoid")(x)

model_gru = Model(sequence_input, outp )
model_gru.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
outputs = []

model_gru_score = model_gru.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=4, batch_size=128,verbose=2)
model_gru_score = model_gru_score.history
model_gru_score = np.array(model_gru_score['val_loss'])
model_gru_score = np.mean(model_gru_score)
print(model_gru_score)
y_val_pred_gru = model_gru.predict(x_val)

outputs.append([y_val_pred_gru, model_gru_score, 'LSTM #1'])

In [ ]:
from keras.layers import GlobalAveragePooling1D,GlobalMaxPooling1D,concatenate

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.1)(conc)
outp = Dense(1, activation="sigmoid")(conc)
    
model_gru_2 = Model(inputs=inp, outputs=outp)
model_gru_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_gru_2_score = model_gru_2.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=3, batch_size=128,verbose=2)
model_gru_2_score = model_gru_2_score.history
model_gru_2_score = np.array(model_gru_2_score['val_loss'])
model_gru_2_score = np.mean(model_gru_2_score)
print(model_gru_2_score)
y_val_pred_gru_2 = model_gru_2.predict(x_val)

outputs.append([y_val_pred_gru_2, model_gru_2_score, 'LSTM #2'])

In [ ]:
from keras.layers import Reshape,Conv2D,MaxPool2D,Concatenate

filter_sizes = [1,2,3,5]
num_filters = 36

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = Reshape((maxlen, embed_size, 1))(x)

maxpool_pool = []
for i in range(len(filter_sizes)):
    conv = Conv2D(num_filters, kernel_size=(filter_sizes[i], embed_size),
                                     kernel_initializer='he_normal', activation='elu')(x)
    maxpool_pool.append(MaxPool2D(pool_size=(maxlen - filter_sizes[i] + 1, 1))(conv))

z = Concatenate(axis=1)(maxpool_pool)   
z = Flatten()(z)
z = Dropout(0.1)(z)
outp = Dense(1, activation="sigmoid")(z)

model_cnn = Model(inputs=inp, outputs=outp)
model_cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_cnn_score = model_cnn.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=4, batch_size=128,verbose=2)
model_cnn_score = model_cnn_score.history
model_cnn_score = np.array(model_cnn_score['val_loss'])
model_cnn_score = np.mean(model_cnn_score)
print(model_cnn_score)
y_val_pred_cnn = model_cnn.predict(x_val)

outputs.append([y_val_pred_cnn, model_cnn_score, 'CNN'])

In [ ]:
inp = Input(shape=(maxlen,))
x = Embedding(max_features,
                            embed_size,
                            weights=[embedding_matrix],
                            input_length=maxlen,
                            trainable=False)(inp)
x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
x = Attention(maxlen)(x) # New
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model_gru_att = Model(inputs=inp, outputs=x)
model_gru_att.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_gru_att_score = model_gru_att.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=4, batch_size=128,verbose=2)
model_gru_att_score = model_gru_att_score.history
model_gru_att_score = np.array(model_gru_att_score['val_loss'])
model_gru_att_score = np.mean(model_gru_att_score)
print(model_gru_att_score)
y_val_pred_gru_att = model_gru_att.predict(x_val)

outputs.append([y_val_pred_gru_att, model_gru_att_score , 'CNN'])

In [ ]:
outputs.sort(key=lambda x: x[1]) # Sort the output by val f1 score
weights = [i for i in range(1, len(outputs) + 1)]
weights = [float(i) / sum(weights) for i in weights] 
print(weights)

In [ ]:
from sklearn.linear_model import LinearRegression
X = np.asarray([outputs[i][0] for i in range(len(outputs))])
X = X[...,0]
reg = LinearRegression().fit(X.T, y_val)
print(reg.score(X.T, y_val),reg.coef_)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve

def threshold_search(y_true, y_proba, plot=False):
    precision, recall, thresholds = precision_recall_curve(y_true, y_proba)
    thresholds = np.append(thresholds, 1.001) 
    F = 2 / (1/precision + 1/recall)
    best_score = np.max(F)
    best_th = thresholds[np.argmax(F)]
    if plot:
        plt.plot(thresholds, F, '-b')
        plt.plot([best_th], [best_score], '*r')
        plt.show()
    search_result = {'threshold': best_th , 'f1': best_score}
    return search_result

#Averaging
y_val_pred = np.array([y_val_pred_gru, y_val_pred_cnn,y_val_pred_gru_2,y_val_pred_gru_att])
y_val_pred = np.average(y_val_pred, axis=0)

#Blending with LR
#y_val_pred = np.sum([outputs[i][0] * reg.coef_[i] for i in range(len(outputs))], axis = 0)

thresh_results = threshold_search(y_val,y_val_pred)
threshold = thresh_results['threshold']
print(thresh_results['threshold'])

In [ ]:
test = pd.read_csv("../input/test.csv")
# TO REMOVE 
#test = test.head(10)
test["question_text"] = test["question_text"].str.lower()
test['question_text'] = test['question_text'].apply(lambda x: clean_special_chars(x, punct, punct_mapping))
test['question_text'] = test['question_text'].apply(lambda x: clean_numbers(x))
test['question_text'] = test['question_text'].apply(lambda x: clean_contractions(x, contraction_mapping))
test_texts = test["question_text"].fillna("_##_").values

test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=maxlen)

y_pred_gru = model_gru.predict(test_data)
y_pred_gru_2 = model_gru.predict(test_data)
y_pred_cnn = model_cnn.predict(test_data)
y_pred_gru_att = model_gru_att.predict(test_data)

test_outputs = [y_pred_gru,y_pred_gru_2,y_pred_cnn,y_pred_gru_att]

#Averaging predictions
y_pred = np.array([y_pred_gru, y_pred_cnn,y_pred_gru_2,y_pred_gru_att])
y_pred = np.average(y_pred, axis=0)

#Blending with LR
#y_pred = np.sum([test_outputs[i] * reg.coef_[i] for i in range(len(outputs))], axis = 0)

y_threshed = (np.array(y_pred) > threshold).astype(np.int)
flat_list = [item for sublist in y_threshed for item in sublist]

submit_df = pd.DataFrame({"qid": test["qid"].values, "prediction": flat_list})
submit_df.to_csv("submission.csv", index=False)